#### Do not run this cell unless you want to download the RT-GTFS from DTC website

In [ ]:
# import requests
# import csv
# import gtfs_realtime_pb2
# import time

# PB_URL = "https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=qQdrLdHL9z5X2IEPMv1JNx524TutmrQV"

# positions = str

# cols = ["entity_id","trip","route","start_time","start_date","vehicle_id","lat","lon","veh_speed","veh_timestamp"]
# iteration = 1
# while True: # Code will run forever without this while statement
#     response = requests.get(PB_URL)
#     response.raise_for_status()

#     feed_message = gtfs_realtime_pb2.FeedMessage()

#     feed_message.ParseFromString(response.content)

#     with open('output/dtc-rt/dtc-rt-gtfs-output.csv','a',newline='') as csv_file:
#         data = []
#         writer = csv.writer(csv_file)

#         trips,vehicle,position,timestamp = None,None,None,None

#         for entity in feed_message.entity:
#             entity_id = entity.id
#             trip = entity.vehicle.trip.trip_id
#             route = entity.vehicle.trip.route_id
#             start_time = entity.vehicle.trip.start_time
#             start_date = entity.vehicle.trip.start_date

#             vehicle_id = entity.vehicle.vehicle.id

#             lat = entity.vehicle.position.latitude
#             lon = entity.vehicle.position.longitude
#             veh_speed = entity.vehicle.position.speed

#             veh_timestamp = entity.vehicle.timestamp
#             # print(entity)
#             data.append([entity_id,trip,route,start_time,start_date,vehicle_id,lat,lon,veh_speed,veh_timestamp])
#         print(f"Iteration No: {iteration}")
#         iteration += 1
#         print(len(data))
#         writer.writerows(data)
#     time.sleep(3)

In [1]:
import pandas as pd
import folium
from datetime import datetime
import random

In [14]:
df = pd.read_csv('output/dtc-rt/dtc-rt-gtfs-output.csv',parse_dates=['start_time','start_date'])

cols = ["entity_id","trip","route","start_time","start_date","vehicle_id","lat","lon","veh_speed","veh_timestamp"]

In [31]:
# Filter only DTC Routes
df = df.loc[df['route']>5000].reset_index()

In [35]:
df['start_datetime'] = pd.to_datetime(pd.to_datetime(df['start_date']).dt.strftime('%Y-%m-%d') + ' ' + pd.to_datetime(df['start_time']).dt.strftime('%H:%M:%S'))

In [37]:
df['veh_timestamp'] = pd.to_datetime(df['veh_timestamp'])

# Group by vehicle_id and sort by start_time
grouped_df = df.sort_values(by=['vehicle_id', 'start_datetime']).groupby('vehicle_id')

In [38]:
def rgb_to_hex():
    color = random.choices(range(150),k=3)

    r = color[0]
    g = color[1]
    b = color[2]
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

In [45]:
max_routes = 5

my_map = folium.Map(None)
grp = folium.FeatureGroup().add_to(my_map)
for route, (name, group) in enumerate(grouped_df, start=1):
    tcolor = rgb_to_hex()
    if route < max_routes:
        for index, row in group.iterrows():
            
            if index % 5 == 0:
                folium.Marker(
                    location=[row['lat'], row['lon']],
                    popup=f"Vehicle ID: {row['vehicle_id']}<br>Start Time: {row['start_time']}<br>Speed: {row['veh_speed']} km/h"
                ).add_to(grp)

In [46]:
my_map.fit_bounds(grp.get_bounds())
my_map